In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import csv
import sys
import json
# df = pd.read_csv('tech_file.csv')
# orig_txt = df['transcript'][25]
import torch
import random

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)


In [2]:
# !python3 -m spacy download en_core_web_sm

In [3]:

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

In [4]:
csv.field_size_limit(sys.maxsize)

podcast_data = []
row_num = 0
with open('../summarize/vtt_data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter='|')
    for row in reader:
        row_num += 1
        
        if row_num == 1:
            continue
            
        filename = row[5]
        if not filename.endswith("_large.vtt"):
            continue

        podcast = {    
            "episode_index": row[0],    
            "guest": row[1],
            "episode_name": row[2],
            "host_name": row[3],
            "episode_number": row[4],
            "transcript": row[6],
            "duration": row[7],
        }
        podcast_data.append(podcast)

In [5]:
# benchmark_summaries = {
#     "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_benchmark.json",
#     "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_benchmark.json",
#     "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_benchmark.json",
#     "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_benchmark.json",
# }

# keypoints_summaries = {
#     "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_stage3_extractkeypoints_v2b4.json",
#     "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_stage3_extractkeypoints_v2b4.json",
#     "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_stage3_extractkeypoints_v2b4.json",
#     "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_stage3_extractkeypoints_v2b4_ep94only.json",
# }

In [6]:

total_benchmark_precision = 0
total_benchmark_recall = 0

total_keypoints_precision = 0
total_keypoints_recall = 0

# 15 more
episodes_number = [
    3,4,5,6,7,9,10,11,13,14,15,17,18,19,20
]

count = 0
for podcast in podcast_data:

    episode_number = podcast['episode_number']
    
    if int(episode_number) not in episodes_number:
        continue
        
    count += 1
    
    print(f"Calculating score for episode: {podcast['episode_number']}")    
    summary = f"../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_{episode_number}_benchmark.json" 
    
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    # Process the texts
    source_doc = nlp(podcast['transcript'])
    
    summary_doc = nlp(summary_json['final_summary'])

    # Extract entities from both texts
    source_entities = set([(ent.text, ent.label_) for ent in source_doc.ents])
    summary_entities = set([(ent.text, ent.label_) for ent in summary_doc.ents])

    # Calculate correct NEs in summary
    correct_entities_in_summary = summary_entities.intersection(source_entities)

    # Calculate Precision and Recall
    precision = len(correct_entities_in_summary) / len(summary_entities) if summary_entities else 0
    recall = len(correct_entities_in_summary) / len(source_entities) if source_entities else 0

    total_benchmark_precision += precision
    total_benchmark_recall += recall
    
    print("Benchmark:")
    print(f"NE Precision: {precision:.2f}")
    print(f"NE Recall: {recall:.2f}")
    
    # keypoints
    
    summary = f"../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_{episode_number}_stage3_extractkeypoints_v2b4.json"
      
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    # Process the texts
    summary_doc = nlp(summary_json['final_summary'])

    # Extract entities from both texts
    summary_entities = set([(ent.text, ent.label_) for ent in summary_doc.ents])

    # Calculate correct NEs in summary
    correct_entities_in_summary = summary_entities.intersection(source_entities)

    # Calculate Precision and Recall
    precision = len(correct_entities_in_summary) / len(summary_entities) if summary_entities else 0
    recall = len(correct_entities_in_summary) / len(source_entities) if source_entities else 0

    total_keypoints_precision += precision
    total_keypoints_recall += recall
    
    print("Keypoints:")
    print(f"NE Precision: {precision:.2f}")
    print(f"NE Recall: {recall:.2f}")
    
    print("==============================================")
    
    
# calculate average precision and recall scores


print(f"Avg Benchmark NE Precision: {total_benchmark_precision/count:.2f}")
print(f"Avg Benchmark NE Recall: {total_benchmark_recall/count:.2f}")

print(f"Avg Keypoints NE Precision: {total_keypoints_precision/count:.2f}")
print(f"Avg Keypoints NE Recall: {total_keypoints_recall/count:.2f}")
    

Calculating score for episode: 3
Benchmark:
NE Precision: 0.67
NE Recall: 0.16
Keypoints:
NE Precision: 0.55
NE Recall: 0.12
Calculating score for episode: 4
Benchmark:
NE Precision: 1.00
NE Recall: 0.04
Keypoints:
NE Precision: 0.87
NE Recall: 0.27
Calculating score for episode: 5
Benchmark:
NE Precision: 0.71
NE Recall: 0.13
Keypoints:
NE Precision: 0.75
NE Recall: 0.19
Calculating score for episode: 6
Benchmark:
NE Precision: 0.76
NE Recall: 0.16
Keypoints:
NE Precision: 0.60
NE Recall: 0.07
Calculating score for episode: 7
Benchmark:
NE Precision: 0.63
NE Recall: 0.06
Keypoints:
NE Precision: 0.77
NE Recall: 0.08
Calculating score for episode: 9
Benchmark:
NE Precision: 0.44
NE Recall: 0.04
Keypoints:
NE Precision: 0.67
NE Recall: 0.10
Calculating score for episode: 10
Benchmark:
NE Precision: 0.80
NE Recall: 0.05
Keypoints:
NE Precision: 0.86
NE Recall: 0.14
Calculating score for episode: 11
Benchmark:
NE Precision: 1.00
NE Recall: 0.05
Keypoints:
NE Precision: 0.73
NE Recall: 0.0